In [344]:
import scrapy

In [542]:
from bs4 import BeautifulSoup
import urllib.request as req
import re,sys,time,datetime

# はじめに...
SSのまとめサイトから  

```html
ゆの「あはは・・・」 

ﾀﾞｯﾀﾞｯﾀﾞｯ ｶﾞﾁｬｯ 

大家「・・・おじゃまします・・・」ｾﾞｪｾﾞｪ 

なずな「あ、大家さん♡こんにちは！」 
```
からキャラとそのキャラのセリフを抽出して以下のようなjson形式で保存する関数を作成しました.  
```json
{　　
character:ゆの　　
speech:あはは・・・　　
}　　
{　　
character:大家　　
speech:・・・おじゃまします・・・　　
}　　
...
```


タグの都合上, 現在は限られたサイトでしか正常に作動しません.  
そのため, 次回作成する際は scrapy を用いて作ってみたいと思ってます♩  


### 作成経緯
日本語の会話データセットで無料配布されているものってほとんどないんですよね.  
英語ではいくつかあるのに  
  
そして, TwitterなどのSNSサイトのリプライを会話のデータセットとして使う場合は不自然な日本語で会話しているため, 会話として成立していない例を多く含むと感じてます.  
そのため, 今回は物語それも会話劇の抽出をやってみました.  
  
この手法は今回のようなSS以外にも"なろう"の文章をダウンロードしてセリフ抽出というのも有効だと思います.  
ただし, 一般小説やなろうのように背景や説明が多くてセリフのみだと話者が誰か不明だったり, 話の流れが不明な場合が多いのが難点かなと思います.  

In [579]:
urlName = "http://yomicom.jp/blog-entry-1095.html"

In [580]:
created_name = str(datetime.datetime.now().timestamp()).split(".")[0]
#urlName = "http://yomicom.jp/blog-entry-1370.html"
#urlName = "http://yomicom.jp/blog-entry-1372.html"
#op = getURL_title(urlName)
#output_file = op+".json"
output_file = created_name+".json"
content , tmpName = web_convert_Text(urlName)
extract_chat(tmpName,output_file)
print("################END###################")

################END###################


In [346]:
def getURL(url):
    res = req.urlopen(url)
    return BeautifulSoup(res,"html.parser")

In [520]:
def getURL_title(url):
    src=getURL(url)
    title = src.select("div.ently_outline > h2.ently_title > a ")
    #re.search('title="(.*)"',title)
    title_pattern = ">(.*)</a>"
    title = str(title).replace("\n","")
    title = title.replace("\r","")
    return re.findall(title_pattern,title,re.S)[0]

In [528]:
def web_convert_Text(urlName):    
    #urlName = "http://yomicom.jp/blog-entry-1372.html"
    #urlName = "http://yomicom.jp/blog-entry-1364.html"

    src = getURL(urlName)

    #出力するファイルはまとめサイトのタイトルにしました.
    op = getURL_title(urlName)
    output_file = op+"json"
    #整形して一時的に保存するテキストファイル
    tmp_file = op+".txt"


    text = src.select("div.ently_text > div.t_b")
    #div内のみ得る正規表現
    pat="<div class=\"t_b\">(.*)</div>"
    div_content = re.compile(pat,re.S)
    matome_list = []
    for line in text:
        #文字列に変換
        test = str(line)
        #<br/>を\nに変換
        test = test.replace("<br/>","\n")
        #print(test)
        test=div_content.findall(test)
        test.append("\n")
        matome_list.append(test)
    #リストを下げる
    flatten = lambda l: [item for sublist in l for item in sublist]


    res = flatten(matome_list)
    #まとめサイトのメインコンテンツをテキストで保存
    f = open(tmp_file,"w")
    for line in res:
        f.write(line)
    f.close()
    return res,tmp_file

In [574]:
import re
import json

#引数: ファイル
def extract_chat(input_file,output_file):
    #pattern
    chat_pattern = r"「(.+?)」"
    speaker_pattern = "(.*)(?=「)"
    
    #ファイルを読む, 
    with open(input_file) as f:
        text = f.read()
    #出力ファイル(json)
    output = open(output_file,'w')
    #「」会話終わりに強制的に改行を加える.  
    #text = text.replace("」","」\n")
    #print(text)
    text = re.sub(r"」(.+?)","」\n",text)
    #print("MEL",text)
    #print(text)
    #regexp
    character = re.compile(speaker_pattern).findall(text)
    speech = re.compile(chat_pattern,re.S).findall(text)
    
    #リスト二含まれている""を削除
    character_ = [c for c in character if c is not ""]
    #改行文字を削除
    speech_ = [test.replace("\n","") for test in speech]
    #print(character_)
    
    #x 返すキャラとそのセリフを辞書に格納
    #o jsonを返す
    c_s = []
    num=0
    for i,j in zip(character_,speech_):
        #print(num,i,j)
        #c_s.update({num:{"character":i,"speech":j}})
        c_s.append({"id":num,"character":i,"speech":j})
        #c_s["character"] = i
        #c_s["speech"] = j
        #print(num,i,j)
        num += 1
        #print(c_s)
    json.dump(c_s,output,ensure_ascii=False,indent = 4,separators=(",",": "))
    return c_s
    #json.dump(c_s,output,ensure_ascii=False,indent = 4,separators=(",",": ")) 

## ファイル読み込み
自分用

In [570]:
## ファイル読み込み
gabdro = "1492687139.json"
f = open(gabdro, 'r')
result = json.load(f)
for i in result:
    print(i["character"].replace(" ",""),"「"+i["speech"]+"」")

ガヴ 「明日は私の誕生日……」
ガヴ 「……眠れない」
ガヴ 「ネトゲ……はやめとくか」
ガヴ 「……とりあえず目を休めよう」
ガヴ 「……そうしよう」
ガヴ 「……結局一睡も出来なかった」
ガヴ 「まあでも……楽しみだな」
ガヴ 「……！はやいな！」
男 「お届け物でーす」
ガヴ 「……だよな」
男 「？」
ガヴ 「……まぁまだ朝だしな」
ガヴ 「……昼か、まあカップラーメンでいいか」
ガヴ 「……もうそろそろ日が沈むぞ」
ガヴ 「……」
ガヴ 「……！！！」
ガヴ 「はっ、はい！」
オバサン 「あっ、こんにちは。宗教に興味はないですか？今なら絶対に幸せに」
ガヴ 「……はぁ、すいません私の存在が宗教ですので」
オバサン 「……は？」
ガヴ 「……遅い」
ガヴ 「……もう来ないのかな」
ガヴ 「……そうだよな、こんな駄天使の誕生日なんて誰が祝……」
ガヴ 「……」
ガヴ 「……もしかして」
 「なーはっはっは！！！」
ガヴ 「……！！！」
 「大悪魔様がきてあげたわよ！」
ガヴ 「はっ、はい！」
サタラフィヴィーネ 「Happy Birthday！」
ヴィーネ 「ガヴ、お誕生日おめでとう！」
ガヴ 「……あ、そうか！今日私の誕生日か！忘れてたわ～」
サターニャ 「ふふっ、昨日夜楽しみで寝られなかったの知ってんだからね」
ガヴ 「なっ、そんなことは」
ラフィ 「うふふ～」
ガヴ 「なっ……！もしかしておまえ千里眼で！」
ヴィーネ 「……とりあえずおめでとう！」
ガヴ 「……ありがとう」
サターニャ 「……素直じゃないわねガヴリールは」
ガヴ 「なっなんだと！」
サターニャ 「なによー」
ラフィ 「まぁまぁ」
ガヴ 「……と、とりあえずお前達も感謝してやる！」
ヴィーネ 「ふふっ、サターニャみたいになってるわよガヴ」
ガヴサタ 「なっ……」
サターニャ 「……まぁいいわ、とりあえずケーキ！ケーキよ！」
ガヴ 「……でかいな」
ヴィーネ 「サターニャが作ったのよ」
ガヴ 「えっ！」
ラフィ 「『私が作ってあげるの！』って言い出したら聞かなくって大変だったんですよ」
サターニャ 「ちょ、ちょっと！それは言わないって約束でしょ！？」
ラフィ 「あら、ごめんなさい♪」
ガヴ 「……ありがとうサターニャ。悪かったな」
サターニャ 「べっ、別にいいわよ！ほらっ、味わって食べなさい